<h3>Crawling Data Tweet</h3>

In [43]:
import tweepy
import pandas as pd

api_key = "pQvbI4r9gL5Bg43vyZ8ysaEjt"
api_secret_key = "f5NRetIdoA5zLfaBwjNZBmOZTXOKUrHc3zo54ZtzjcFPuoftXB"
access_token = "1133613434824609792-pdaCDmG5dERjBQwaRzZuNylYZw8F2g"
access_token_secret = "nUwiQEXaErJzZ6lRI5ULtBEqOyKPoUBOQgYh0I5zjxqMA"

auth = tweepy.OAuthHandler(api_key, api_secret_key)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit=True)


In [44]:
tweet_count = 300
search_words = "#covid-19"
date = "2021-01-13"

tweets = tweepy.Cursor(api.search, q=search_words, lang="id", since = date, tweet_mode='extended').items(tweet_count)
tweet_list = [tweet for tweet in tweets]

In [45]:
result=[]
for tweet in tweet_list:  
    try:
        text = tweet.retweeted_status.full_text
    except AttributeError:  # Not a Retweet
        text = tweet.full_text
    ith_tweet = [tweet.user.screen_name, tweet.user.location, tweet.created_at, tweet.retweet_count, text.replace('\n', ' ')]
    if ith_tweet not in result:
        result.append(ith_tweet)
        
df_tweet=pd.DataFrame(result, columns=['username', 'location', 'tweetdate', 'retweetcount', 'text'])
# df_tweet.style.hide_index()

In [46]:
#export
df_tweet.to_csv('hasil_twitter_scraping.csv', index = False)
print('scraping selesai!')

scraping selesai!
scraping selesai!
scraping selesai!
scraping selesai!


<h3>Preprocessing</h3>

In [47]:
import pandas as pd
datatweet=pd.read_csv('hasil_twitter_scraping.csv').get('text')
df=pd.DataFrame(datatweet)
df.style.hide_index()

text
"""Saya tidak sokong golongan antivaksin ini tapi adakalanya hujah mereka agak kuat, jadi kita perlu balas mereka dengan hujah yang lebih kukuh."" - Anwar Ibrahim @anwaribrahim https://t.co/MIalP4EaPZ"
"Negara merekodkan 6,145 jangkitan baharu wabak COVID-19 dalam tempoh 24 jam lalu. Angka yang direkodkan @KKMPutrajaya hari ini merupakan jangkitan harian terendah dalam tempoh lebih tiga bulan lalu. Ikuti berita terkini di https://t.co/Ia2kVam8YD #AWANInews https://t.co/Nat6LCofPr"
"Negara merekodkan 6,145 jangkitan baharu wabak COVID-19 dalam tempoh 24 jam lalu. Angka yang direkodkan @KKMPutrajaya hari ini merupakan jangkitan harian terendah dalam tempoh lebih tiga bulan lalu. Ikuti berita terkini di https://t.co/Ia2kVam8YD #AWANInews https://t.co/Nat6LCofPr"
"Negara merekodkan 6,145 jangkitan baharu wabak COVID-19 dalam tempoh 24 jam lalu. Angka yang direkodkan @KKMPutrajaya hari ini merupakan jangkitan harian terendah dalam tempoh lebih tiga bulan lalu. Ikuti berita terkini di https://t.co/Ia2kVam8YD #AWANInews https://t.co/Nat6LCofPr"
"COVID-19 | 6,145 kes baharu dilaporkan dengan kes kumulatif 2,390,687. https://t.co/ap95OPcDRN"
"Negara merekodkan 6,145 jangkitan baharu wabak COVID-19 dalam tempoh 24 jam lalu. Angka yang direkodkan @KKMPutrajaya hari ini merupakan jangkitan harian terendah dalam tempoh lebih tiga bulan lalu. Ikuti berita terkini di https://t.co/Ia2kVam8YD #AWANInews https://t.co/Nat6LCofPr"
"Khairy turut menegaskan, Kementerian Kesihatan berkemungkinan mewajibkan golongan yang menolak atau melengahkan pengambilan vaksin untuk menjalankan ujian calit Covid-19 secara mingguan dan kerajaan tidak akan menanggung perbelanjaan itu. https://t.co/l2iTf3Y99H"
"Negara merekodkan 6,145 jangkitan baharu wabak COVID-19 dalam tempoh 24 jam lalu. Angka yang direkodkan @KKMPutrajaya hari ini merupakan jangkitan harian terendah dalam tempoh lebih tiga bulan lalu. Ikuti berita terkini di https://t.co/Ia2kVam8YD #AWANInews https://t.co/Nat6LCofPr"
"COVID-19 | 6,145 kes baharu dilaporkan dengan kes kumulatif 2,390,687. https://t.co/ap95OPcDRN"
“Maklumat terperinci akan dimuatnaik ke laman COVIDNOW: https://t.co/P2cHWIifbx. https://t.co/sTXCiDywSl


text
"""Saya tidak sokong golongan antivaksin ini tapi adakalanya hujah mereka agak kuat, jadi kita perlu balas mereka dengan hujah yang lebih kukuh."" - Anwar Ibrahim @anwaribrahim https://t.co/MIalP4EaPZ"
"Negara merekodkan 6,145 jangkitan baharu wabak COVID-19 dalam tempoh 24 jam lalu. Angka yang direkodkan @KKMPutrajaya hari ini merupakan jangkitan harian terendah dalam tempoh lebih tiga bulan lalu. Ikuti berita terkini di https://t.co/Ia2kVam8YD #AWANInews https://t.co/Nat6LCofPr"
"Negara merekodkan 6,145 jangkitan baharu wabak COVID-19 dalam tempoh 24 jam lalu. Angka yang direkodkan @KKMPutrajaya hari ini merupakan jangkitan harian terendah dalam tempoh lebih tiga bulan lalu. Ikuti berita terkini di https://t.co/Ia2kVam8YD #AWANInews https://t.co/Nat6LCofPr"
"Negara merekodkan 6,145 jangkitan baharu wabak COVID-19 dalam tempoh 24 jam lalu. Angka yang direkodkan @KKMPutrajaya hari ini merupakan jangkitan harian terendah dalam tempoh lebih tiga bulan lalu. Ikuti berita terkini di https://t.co/Ia2kVam8YD #AWANInews https://t.co/Nat6LCofPr"
"COVID-19 | 6,145 kes baharu dilaporkan dengan kes kumulatif 2,390,687. https://t.co/ap95OPcDRN"
"Negara merekodkan 6,145 jangkitan baharu wabak COVID-19 dalam tempoh 24 jam lalu. Angka yang direkodkan @KKMPutrajaya hari ini merupakan jangkitan harian terendah dalam tempoh lebih tiga bulan lalu. Ikuti berita terkini di https://t.co/Ia2kVam8YD #AWANInews https://t.co/Nat6LCofPr"
"Khairy turut menegaskan, Kementerian Kesihatan berkemungkinan mewajibkan golongan yang menolak atau melengahkan pengambilan vaksin untuk menjalankan ujian calit Covid-19 secara mingguan dan kerajaan tidak akan menanggung perbelanjaan itu. https://t.co/l2iTf3Y99H"
"Negara merekodkan 6,145 jangkitan baharu wabak COVID-19 dalam tempoh 24 jam lalu. Angka yang direkodkan @KKMPutrajaya hari ini merupakan jangkitan harian terendah dalam tempoh lebih tiga bulan lalu. Ikuti berita terkini di https://t.co/Ia2kVam8YD #AWANInews https://t.co/Nat6LCofPr"
"COVID-19 | 6,145 kes baharu dilaporkan dengan kes kumulatif 2,390,687. https://t.co/ap95OPcDRN"
“Maklumat terperinci akan dimuatnaik ke laman COVIDNOW: https://t.co/P2cHWIifbx. https://t.co/sTXCiDywSl


text
"""Saya tidak sokong golongan antivaksin ini tapi adakalanya hujah mereka agak kuat, jadi kita perlu balas mereka dengan hujah yang lebih kukuh."" - Anwar Ibrahim @anwaribrahim https://t.co/MIalP4EaPZ"
"Negara merekodkan 6,145 jangkitan baharu wabak COVID-19 dalam tempoh 24 jam lalu. Angka yang direkodkan @KKMPutrajaya hari ini merupakan jangkitan harian terendah dalam tempoh lebih tiga bulan lalu. Ikuti berita terkini di https://t.co/Ia2kVam8YD #AWANInews https://t.co/Nat6LCofPr"
"Negara merekodkan 6,145 jangkitan baharu wabak COVID-19 dalam tempoh 24 jam lalu. Angka yang direkodkan @KKMPutrajaya hari ini merupakan jangkitan harian terendah dalam tempoh lebih tiga bulan lalu. Ikuti berita terkini di https://t.co/Ia2kVam8YD #AWANInews https://t.co/Nat6LCofPr"
"Negara merekodkan 6,145 jangkitan baharu wabak COVID-19 dalam tempoh 24 jam lalu. Angka yang direkodkan @KKMPutrajaya hari ini merupakan jangkitan harian terendah dalam tempoh lebih tiga bulan lalu. Ikuti berita terkini di https://t.co/Ia2kVam8YD #AWANInews https://t.co/Nat6LCofPr"
"COVID-19 | 6,145 kes baharu dilaporkan dengan kes kumulatif 2,390,687. https://t.co/ap95OPcDRN"
"Negara merekodkan 6,145 jangkitan baharu wabak COVID-19 dalam tempoh 24 jam lalu. Angka yang direkodkan @KKMPutrajaya hari ini merupakan jangkitan harian terendah dalam tempoh lebih tiga bulan lalu. Ikuti berita terkini di https://t.co/Ia2kVam8YD #AWANInews https://t.co/Nat6LCofPr"
"Khairy turut menegaskan, Kementerian Kesihatan berkemungkinan mewajibkan golongan yang menolak atau melengahkan pengambilan vaksin untuk menjalankan ujian calit Covid-19 secara mingguan dan kerajaan tidak akan menanggung perbelanjaan itu. https://t.co/l2iTf3Y99H"
"Negara merekodkan 6,145 jangkitan baharu wabak COVID-19 dalam tempoh 24 jam lalu. Angka yang direkodkan @KKMPutrajaya hari ini merupakan jangkitan harian terendah dalam tempoh lebih tiga bulan lalu. Ikuti berita terkini di https://t.co/Ia2kVam8YD #AWANInews https://t.co/Nat6LCofPr"
"COVID-19 | 6,145 kes baharu dilaporkan dengan kes kumulatif 2,390,687. https://t.co/ap95OPcDRN"
“Maklumat terperinci akan dimuatnaik ke laman COVIDNOW: https://t.co/P2cHWIifbx. https://t.co/sTXCiDywSl


text
"""Saya tidak sokong golongan antivaksin ini tapi adakalanya hujah mereka agak kuat, jadi kita perlu balas mereka dengan hujah yang lebih kukuh."" - Anwar Ibrahim @anwaribrahim https://t.co/MIalP4EaPZ"
"Negara merekodkan 6,145 jangkitan baharu wabak COVID-19 dalam tempoh 24 jam lalu. Angka yang direkodkan @KKMPutrajaya hari ini merupakan jangkitan harian terendah dalam tempoh lebih tiga bulan lalu. Ikuti berita terkini di https://t.co/Ia2kVam8YD #AWANInews https://t.co/Nat6LCofPr"
"Negara merekodkan 6,145 jangkitan baharu wabak COVID-19 dalam tempoh 24 jam lalu. Angka yang direkodkan @KKMPutrajaya hari ini merupakan jangkitan harian terendah dalam tempoh lebih tiga bulan lalu. Ikuti berita terkini di https://t.co/Ia2kVam8YD #AWANInews https://t.co/Nat6LCofPr"
"Negara merekodkan 6,145 jangkitan baharu wabak COVID-19 dalam tempoh 24 jam lalu. Angka yang direkodkan @KKMPutrajaya hari ini merupakan jangkitan harian terendah dalam tempoh lebih tiga bulan lalu. Ikuti berita terkini di https://t.co/Ia2kVam8YD #AWANInews https://t.co/Nat6LCofPr"
"COVID-19 | 6,145 kes baharu dilaporkan dengan kes kumulatif 2,390,687. https://t.co/ap95OPcDRN"
"Negara merekodkan 6,145 jangkitan baharu wabak COVID-19 dalam tempoh 24 jam lalu. Angka yang direkodkan @KKMPutrajaya hari ini merupakan jangkitan harian terendah dalam tempoh lebih tiga bulan lalu. Ikuti berita terkini di https://t.co/Ia2kVam8YD #AWANInews https://t.co/Nat6LCofPr"
"Khairy turut menegaskan, Kementerian Kesihatan berkemungkinan mewajibkan golongan yang menolak atau melengahkan pengambilan vaksin untuk menjalankan ujian calit Covid-19 secara mingguan dan kerajaan tidak akan menanggung perbelanjaan itu. https://t.co/l2iTf3Y99H"
"Negara merekodkan 6,145 jangkitan baharu wabak COVID-19 dalam tempoh 24 jam lalu. Angka yang direkodkan @KKMPutrajaya hari ini merupakan jangkitan harian terendah dalam tempoh lebih tiga bulan lalu. Ikuti berita terkini di https://t.co/Ia2kVam8YD #AWANInews https://t.co/Nat6LCofPr"
"COVID-19 | 6,145 kes baharu dilaporkan dengan kes kumulatif 2,390,687. https://t.co/ap95OPcDRN"
“Maklumat terperinci akan dimuatnaik ke laman COVIDNOW: https://t.co/P2cHWIifbx. https://t.co/sTXCiDywSl


In [48]:
slangword={}
with open('slangwords.txt') as f:
    lines=f.read().split('\n')
    for kata in lines:
        kata=kata.split(':')
        slangword[kata[0]]=kata[-1]

In [49]:
with open('stopwords.txt') as f:
    more_stopword=f.read().split('\n')

In [50]:
import re
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory, ArrayDictionary
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory, StopWordRemover

SWfactory = StopWordRemoverFactory()
stopword_data = ArrayDictionary(more_stopword+SWfactory.get_stop_words())
stopword = StopWordRemover(stopword_data)
Sfactory = StemmerFactory()
stemmer = Sfactory.create_stemmer()

def praproses(text):
    clean = re.sub("#[^\W]+|@[^\W]+|http[^*\s]+|<[^>]*>|[0-9]|RT", '', text) 
    emoticons = re.findall('(?::|;|=)()(?:-)?(?:\)|\(|D|P)', clean)
    text = (re.sub('[\W]+', ' ', clean.lower()) + 
            ' '.join(emoticons).replace('-', ''))
    chain=''
    for kata in text.split():
        kata=slangword[kata] if kata in slangword.keys() else kata    
        stop = stopword.remove(kata)
        stem = stemmer.stem(stop) 
        chain += f"{stem} " if kata else ''
    return chain

In [51]:
tweet=[]
for kalimat in datatweet:
    result=praproses(kalimat)
    if result not in tweet:
        tweet.append(result)
df=pd.DataFrame(data=tweet,columns=['tweet'])
df.style.hide_index()
#export


tweet
sokong golong antivaksin adakalanya hujah kuat balas hujah kukuh anwar ibrahim
negara merekodkan jangkit baharu wabak covid tempoh jam angka direkodkan jangkit hari rendah tempoh ikut berita kini
covid kes baharu lapor kes kumulatif
khairy tegas menteri kesihatan mungkin wajib golong tolak lengah ambil vaksin jalan uji calit covid minggu raja tanggung belanja
maklumat perinci dimuatnaik laman covidnow
moga angka turun
tip ex data nya
antibodi ngawur org suntik vaksin covid penuh kebal covid vaksin tsb hasil antibodi antibodi hasil vaksin covid ngawur utk covid lihat
harga lantai ibadah umrah kini paras rm alami tingkat kos susul covid
mei ponakan sy tomrowsel sitompul rawat rs advent cihampelas bandung krn papar covid sana jatuh cinta dgn suster rawat wanita puja pasuk berkat nikah pesta nikah covid tdk bicara duka praise god


tweet
sokong golong antivaksin adakalanya hujah kuat balas hujah kukuh anwar ibrahim
negara merekodkan jangkit baharu wabak covid tempoh jam angka direkodkan jangkit hari rendah tempoh ikut berita kini
covid kes baharu lapor kes kumulatif
khairy tegas menteri kesihatan mungkin wajib golong tolak lengah ambil vaksin jalan uji calit covid minggu raja tanggung belanja
maklumat perinci dimuatnaik laman covidnow
moga angka turun
tip ex data nya
antibodi ngawur org suntik vaksin covid penuh kebal covid vaksin tsb hasil antibodi antibodi hasil vaksin covid ngawur utk covid lihat
harga lantai ibadah umrah kini paras rm alami tingkat kos susul covid
mei ponakan sy tomrowsel sitompul rawat rs advent cihampelas bandung krn papar covid sana jatuh cinta dgn suster rawat wanita puja pasuk berkat nikah pesta nikah covid tdk bicara duka praise god


tweet
sokong golong antivaksin adakalanya hujah kuat balas hujah kukuh anwar ibrahim
negara merekodkan jangkit baharu wabak covid tempoh jam angka direkodkan jangkit hari rendah tempoh ikut berita kini
covid kes baharu lapor kes kumulatif
khairy tegas menteri kesihatan mungkin wajib golong tolak lengah ambil vaksin jalan uji calit covid minggu raja tanggung belanja
maklumat perinci dimuatnaik laman covidnow
moga angka turun
tip ex data nya
antibodi ngawur org suntik vaksin covid penuh kebal covid vaksin tsb hasil antibodi antibodi hasil vaksin covid ngawur utk covid lihat
harga lantai ibadah umrah kini paras rm alami tingkat kos susul covid
mei ponakan sy tomrowsel sitompul rawat rs advent cihampelas bandung krn papar covid sana jatuh cinta dgn suster rawat wanita puja pasuk berkat nikah pesta nikah covid tdk bicara duka praise god


tweet
sokong golong antivaksin adakalanya hujah kuat balas hujah kukuh anwar ibrahim
negara merekodkan jangkit baharu wabak covid tempoh jam angka direkodkan jangkit hari rendah tempoh ikut berita kini
covid kes baharu lapor kes kumulatif
khairy tegas menteri kesihatan mungkin wajib golong tolak lengah ambil vaksin jalan uji calit covid minggu raja tanggung belanja
maklumat perinci dimuatnaik laman covidnow
moga angka turun
tip ex data nya
antibodi ngawur org suntik vaksin covid penuh kebal covid vaksin tsb hasil antibodi antibodi hasil vaksin covid ngawur utk covid lihat
harga lantai ibadah umrah kini paras rm alami tingkat kos susul covid
mei ponakan sy tomrowsel sitompul rawat rs advent cihampelas bandung krn papar covid sana jatuh cinta dgn suster rawat wanita puja pasuk berkat nikah pesta nikah covid tdk bicara duka praise god


In [52]:
df.to_csv('clean.csv', index = False, sep=';')

<h3>Bag of Words</h3>

In [53]:
data=pd.read_csv('clean.csv')
df=pd.DataFrame(data)
df.style.hide_index()

tweet
sokong golong antivaksin adakalanya hujah kuat balas hujah kukuh anwar ibrahim
negara merekodkan jangkit baharu wabak covid tempoh jam angka direkodkan jangkit hari rendah tempoh ikut berita kini
covid kes baharu lapor kes kumulatif
khairy tegas menteri kesihatan mungkin wajib golong tolak lengah ambil vaksin jalan uji calit covid minggu raja tanggung belanja
maklumat perinci dimuatnaik laman covidnow
moga angka turun
tip ex data nya
antibodi ngawur org suntik vaksin covid penuh kebal covid vaksin tsb hasil antibodi antibodi hasil vaksin covid ngawur utk covid lihat
harga lantai ibadah umrah kini paras rm alami tingkat kos susul covid
mei ponakan sy tomrowsel sitompul rawat rs advent cihampelas bandung krn papar covid sana jatuh cinta dgn suster rawat wanita puja pasuk berkat nikah pesta nikah covid tdk bicara duka praise god


tweet
sokong golong antivaksin adakalanya hujah kuat balas hujah kukuh anwar ibrahim
negara merekodkan jangkit baharu wabak covid tempoh jam angka direkodkan jangkit hari rendah tempoh ikut berita kini
covid kes baharu lapor kes kumulatif
khairy tegas menteri kesihatan mungkin wajib golong tolak lengah ambil vaksin jalan uji calit covid minggu raja tanggung belanja
maklumat perinci dimuatnaik laman covidnow
moga angka turun
tip ex data nya
antibodi ngawur org suntik vaksin covid penuh kebal covid vaksin tsb hasil antibodi antibodi hasil vaksin covid ngawur utk covid lihat
harga lantai ibadah umrah kini paras rm alami tingkat kos susul covid
mei ponakan sy tomrowsel sitompul rawat rs advent cihampelas bandung krn papar covid sana jatuh cinta dgn suster rawat wanita puja pasuk berkat nikah pesta nikah covid tdk bicara duka praise god


tweet
sokong golong antivaksin adakalanya hujah kuat balas hujah kukuh anwar ibrahim
negara merekodkan jangkit baharu wabak covid tempoh jam angka direkodkan jangkit hari rendah tempoh ikut berita kini
covid kes baharu lapor kes kumulatif
khairy tegas menteri kesihatan mungkin wajib golong tolak lengah ambil vaksin jalan uji calit covid minggu raja tanggung belanja
maklumat perinci dimuatnaik laman covidnow
moga angka turun
tip ex data nya
antibodi ngawur org suntik vaksin covid penuh kebal covid vaksin tsb hasil antibodi antibodi hasil vaksin covid ngawur utk covid lihat
harga lantai ibadah umrah kini paras rm alami tingkat kos susul covid
mei ponakan sy tomrowsel sitompul rawat rs advent cihampelas bandung krn papar covid sana jatuh cinta dgn suster rawat wanita puja pasuk berkat nikah pesta nikah covid tdk bicara duka praise god


tweet
sokong golong antivaksin adakalanya hujah kuat balas hujah kukuh anwar ibrahim
negara merekodkan jangkit baharu wabak covid tempoh jam angka direkodkan jangkit hari rendah tempoh ikut berita kini
covid kes baharu lapor kes kumulatif
khairy tegas menteri kesihatan mungkin wajib golong tolak lengah ambil vaksin jalan uji calit covid minggu raja tanggung belanja
maklumat perinci dimuatnaik laman covidnow
moga angka turun
tip ex data nya
antibodi ngawur org suntik vaksin covid penuh kebal covid vaksin tsb hasil antibodi antibodi hasil vaksin covid ngawur utk covid lihat
harga lantai ibadah umrah kini paras rm alami tingkat kos susul covid
mei ponakan sy tomrowsel sitompul rawat rs advent cihampelas bandung krn papar covid sana jatuh cinta dgn suster rawat wanita puja pasuk berkat nikah pesta nikah covid tdk bicara duka praise god


In [54]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
count = CountVectorizer()

In [55]:
docs = np.array(data['tweet'])
bow = count.fit_transform(docs)
# print(count.vocabulary_)

In [56]:
kata=count.get_feature_names()

In [57]:
#Bag of Words
df=pd.DataFrame(columns=kata, data=bow.toarray())
df

,ab,aba,abai,abdul,abo,achmad,adakalanya,adat,adik,adlh,...,wo,woro,xiv,yah,yatim,yuk,ziarah,zo,zuckerberg,zyada
0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
125,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
126,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
127,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


,ab,aba,abai,abdul,abo,achmad,adakalanya,adat,adik,adlh,...,wo,woro,xiv,yah,yatim,yuk,ziarah,zo,zuckerberg,zyada
0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
125,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
126,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
127,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


,ab,aba,abai,abdul,abo,achmad,adakalanya,adat,adik,adlh,...,wo,woro,xiv,yah,yatim,yuk,ziarah,zo,zuckerberg,zyada
0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
125,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
126,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
127,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


,ab,aba,abai,abdul,abo,achmad,adakalanya,adat,adik,adlh,...,wo,woro,xiv,yah,yatim,yuk,ziarah,zo,zuckerberg,zyada
0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
125,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
126,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
127,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [58]:
#export
df.to_csv('bag_of_word.csv', index = False, sep=';')
print('bag of word!')

bag of word!
bag of word!
bag of word!
bag of word!
